In each data fold, there is a raw data subfolder and a syn data subfolder, which represent the raw data collection without synchronisation but with high precise timestep, and the synchronised data but without high precise timestep.

Here is the header of the sensor file and ground truth file.

## vicon (vi*.csv)

Time  Header  translation.x translation.y translation.z rotation.x rotation.y rotation.z rotation.w

## Sensors (imu*.csv)

Time attitude_roll(radians) attitude_pitch(radians) attitude_yaw(radians) rotation_rate_x(radians/s) rotation_rate_y(radians/s) rotation_rate_z(radians/s) gravity_x(G) gravity_y(G) gravity_z(G) user_acc_x(G) user_acc_y(G) user_acc_z(G) magnetic_field_x(microteslas) magnetic_field_y(microteslas) magnetic_field_z(microteslas)

## Structure

In this folder
```
user@fedora ~/C/magnetic_localization (master)> ls data/Oxford\ Inertial\ Odometry\ Dataset/handheld/data1/syn/
imu1.csv*  imu4.csv*  imu7.csv*  vi3.csv*  vi6.csv*
imu2.csv*  imu5.csv*  vi1.csv*   vi4.csv*  vi7.csv*
imu3.csv*  imu6.csv*  vi2.csv*   vi5.csv*
user@fedora ~/C/magnetic_localization (master)> ls data/Oxford\ Inertial\ Odometry\ Dataset/handheld/data2/syn/
imu1.csv*  imu2.csv*  imu3.csv*  vi1.csv*  vi2.csv*  vi3.csv*
user@fedora ~/C/magnetic_localization (master)> ls data/Oxford\ Inertial\ Odometry\ Dataset/handheld/
data1/  data3/  data5/          Test.txt*
data2/  data4/  handheld.xlsx*  Train.txt*
user@fedora ~/C/magnetic_localization (master)> pwd
/home/user/Code/magnetic_localization
```

Also like each of them are the same length, so no need to sync the timesteps
```
user@fedora ~/C/magnetic_localization (master)> cat data/Oxford\ Inertial\ Odometry\ Dataset/handheld/data1/syn/imu2.csv |wc
  23446   23446 3282548
user@fedora ~/C/magnetic_localization (master)> cat data/Oxford\ Inertial\ Odometry\ Dataset/handheld/data1/syn/vi2.csv |wc
  23446   23446 1740520
```

Just fucking ignore the Time and Header

## Some more Information(exact structure)
```
user@fedora ~/C/m/d/O/h/d/syn (master)> pwd
/home/user/Code/magnetic_localization/data/Oxford Inertial Odometry Dataset/handheld/data1/syn
user@fedora ~/C/m/d/O/h/d/syn (master)> ls
imu1.csv*  imu4.csv*  imu7.csv*  vi3.csv*  vi6.csv*
imu2.csv*  imu5.csv*  vi1.csv*   vi4.csv*  vi7.csv*
imu3.csv*  imu6.csv*  vi2.csv*   vi5.csv*
user@fedora ~/C/m/d/O/h/d/syn (master)> cat imu1.csv |head -n 1
1.50E+11,0.003649,0.44925,-0.21255,0.036483,-0.029496,0.020632,0.003286,-0.43429,-0.90077,-0.002798,0.014599,-0.016836,-2.3577,-0.3717,-42.347
user@fedora ~/C/m/d/O/h/d/syn (master)> cat vi1.csv |head -n 1
1.50E+11,12978,-1.2991,1.7212,1.1931,-0.21409,-0.012459,-0.097183,0.97189
user@fedora ~/C/m/d/O/h/d/syn (master)> 
```

## Goal

Our goal is to predict the current `x`, `y`, `z` based on the previous all previous data(but not previous `x`, `y`, `z`)